In [7]:
%cd /Users/vitthal/Documents/Research/DataScience/MedicalResearch/mgmt

/Users/vitthal/Documents/Research/DataScience/MedicalResearch/mgmt


In [8]:
path = '/Users/vitthal/Documents/Research/DataScience/MedicalResearch/mgmt/'
block_h, block_w = (150, 150)
Sub_Model = 5


In [9]:
# Function Definition --> Loading preprocessed data from the file & Normalize it.

def Load_Data():
    print('Loading Data...')
    pickle_in = open(Split_Path + 'X.pickle', 'rb')
    X = pickle.load(pickle_in)  # Loading Features

    pickle_in = open(Split_Path + 'y.pickle', 'rb')
    y = pickle.load(pickle_in)  # Loading Labels

    print('Shape of X:\t', X.shape), print('Shape of y:\t', y.shape)

    # Normalizing the features between [0, 1]
    #X = X / 255.0

    return X, y


In [10]:
# Function Definition --> For Development, Compilation, Training & Storing of the Model

def Model(img_h, img_w, X_train, y_train, X_test, y_test, count):
    from keras.models import Sequential
    from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, LeakyReLU, Activation
    from keras.utils.np_utils import to_categorical
    model = Sequential()

    model.add(Conv2D(32, (3, 3), padding='same',
              input_shape=(img_h, img_w, 1)))
    model.add(LeakyReLU(alpha=0.1))
    # Pool_size is the size of filter.
    model.add(MaxPooling2D(pool_size=(3, 3)))
    model.add(Dropout(0.1))

    model.add(Conv2D(48, (3, 3), padding='same'))
    model.add(LeakyReLU(alpha=0.1))
    model.add(MaxPooling2D(pool_size=(3, 3)))
    model.add(Dropout(0.1))

    model.add(Conv2D(48, (3, 3), padding='same'))
    model.add(LeakyReLU(alpha=0.1))
    model.add(MaxPooling2D(pool_size=(3, 3)))
    model.add(Dropout(0.1))

    model.add(Flatten())  # Convert 3D feature map to 1D feature vector.

    model.add(Dense(1096))
    model.add(LeakyReLU(alpha=0.1))
    model.add(Dropout(0.1))
    model.add(Dense(2, activation='softmax'))
    # print(model.summary())

    # Compiling the model
    model.compile(loss='categorical_crossentropy',
                  optimizer='adam', metrics=['accuracy'])
    model.optimizer.learning_rate = 0.0001

    # Training the model & storing its detail in the variable h
    history = model.fit(X_train, to_categorical(y_train), batch_size=128, epochs=50, verbose=2,
                        validation_data=(X_test, to_categorical(y_test)), shuffle=True)

    # Evaluating performance of the Model
    test_loss, test_acc = model.evaluate(X_test, to_categorical(y_test))
    results.append(test_acc)
    print('Test Loss is: ', test_loss, '\nTest Accuracy is: ', test_acc)

    # Saving the model in hierarchical Data Formate (HDF)
    model.save(path + 'Outputs/BRATS/flair_output_new/Model_' +
               case + '_' + str(count+1) + '.h5')

    return history, model


In [11]:
# Function Definition --> For Saving Validation Accuracy in the file

def Saving_Test_Accuracy():
    import numpy as np

    print('Saving Test Accuracy...')

    file = open(Result_File, "a")
    file.write(case)
    file.write(':\n======')
    file.write('\nAll Test Accuracy:\t')

    for ans in results:
        file.write(str(ans) + ', ')

    file.write('\nMean Test Accuracy:\t' + str(np.mean(results)))
    file.write('\nMin Test Accuracy:\t' + str(np.min(results)))
    file.write('\nMax Test Accuracy:\t' + str(np.max(results)))
    file.write('\n\n\n')
    file.close()


In [12]:
# All Function Calls
import os
from tqdm import tqdm
import pickle
from sklearn.model_selection import train_test_split
from random import randint

PATH = path + 'Data/BRATS/mod_data1/'
Work_Dir = PATH + 'Working_Data/'
Result_File = path + 'Outputs/BRATS/flair_output_new/Results.txt'
workdir = os.listdir(Work_Dir)
if '.DS_Store' in workdir:
    workdir.remove('.DS_Store')
    print('Removed .DS_Store')
# Calling Functions:
for case in tqdm(workdir):
    Case_Path = os.path.join(Work_Dir, case)  # Joining path for "Cases'.
    # Joining "train" folder with "Cases" path.
    Split_Path = Case_Path + '/train/'
    print(case)

    # Function Call --> Loading preprocessed data from the file & Normalize it.
    X, y = Load_Data()

    results = []
    for i in range(Sub_Model):
        print('\n' + case + '_' + str(i+1))

        # Spliting data into two parts, "Training = 70%" and "Testing = 30%".
        num = randint(0, 1000)
        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=0.3, random_state=num)

        # Function Call --> For Development, Compilation, Training & Storing of the Model
        history, model = Model(block_h, block_w, X_train,
                               y_train, X_test, y_test, i)
    print('\n')

    # Function Call --> For Saving Validation Accuracy in the file
    Saving_Test_Accuracy()
print('Done...!!')
os.system('say "Models are ready"')


Removed .DS_Store


  0%|          | 0/5 [00:00<?, ?it/s]

Case_5
Loading Data...
Shape of X:	 (20694, 150, 150, 1)
Shape of y:	 (20694,)

Case_5_1
Epoch 1/50


2022-11-18 23:21:35.194819: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-11-18 23:21:44.668592: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


114/114 - 11s - loss: 3.5452 - accuracy: 0.5071 - val_loss: 0.8774 - val_accuracy: 0.5149 - 11s/epoch - 99ms/step
Epoch 2/50
114/114 - 10s - loss: 1.1722 - accuracy: 0.5222 - val_loss: 0.7491 - val_accuracy: 0.5457 - 10s/epoch - 86ms/step
Epoch 3/50
114/114 - 10s - loss: 0.9146 - accuracy: 0.5348 - val_loss: 0.8080 - val_accuracy: 0.5270 - 10s/epoch - 87ms/step
Epoch 4/50
114/114 - 10s - loss: 0.8352 - accuracy: 0.5388 - val_loss: 0.7999 - val_accuracy: 0.5326 - 10s/epoch - 85ms/step
Epoch 5/50
114/114 - 10s - loss: 0.8006 - accuracy: 0.5372 - val_loss: 0.7733 - val_accuracy: 0.5384 - 10s/epoch - 87ms/step
Epoch 6/50
114/114 - 10s - loss: 0.7504 - accuracy: 0.5522 - val_loss: 0.9095 - val_accuracy: 0.5183 - 10s/epoch - 85ms/step
Epoch 7/50
114/114 - 10s - loss: 0.7242 - accuracy: 0.5657 - val_loss: 0.8102 - val_accuracy: 0.5268 - 10s/epoch - 85ms/step
Epoch 8/50
114/114 - 10s - loss: 0.7230 - accuracy: 0.5578 - val_loss: 0.7943 - val_accuracy: 0.5239 - 10s/epoch - 85ms/step
Epoch 9/50


...

2022-11-19 02:39:52.589926: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-11-19 02:40:02.787616: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


115/115 - 12s - loss: 3.6017 - accuracy: 0.5101 - val_loss: 0.7929 - val_accuracy: 0.5291 - 12s/epoch - 105ms/step
Epoch 2/50
115/115 - 10s - loss: 1.1315 - accuracy: 0.5213 - val_loss: 0.7207 - val_accuracy: 0.5502 - 10s/epoch - 88ms/step
Epoch 3/50
115/115 - 10s - loss: 0.9508 - accuracy: 0.5315 - val_loss: 0.7203 - val_accuracy: 0.5526 - 10s/epoch - 89ms/step
Epoch 4/50
115/115 - 10s - loss: 0.8376 - accuracy: 0.5385 - val_loss: 0.7216 - val_accuracy: 0.5376 - 10s/epoch - 88ms/step
Epoch 5/50
115/115 - 10s - loss: 0.7809 - accuracy: 0.5503 - val_loss: 0.7600 - val_accuracy: 0.5258 - 10s/epoch - 89ms/step
Epoch 6/50
115/115 - 10s - loss: 0.7502 - accuracy: 0.5512 - val_loss: 0.7074 - val_accuracy: 0.5385 - 10s/epoch - 89ms/step
Epoch 7/50
115/115 - 10s - loss: 0.7378 - accuracy: 0.5497 - val_loss: 0.7466 - val_accuracy: 0.5270 - 10s/epoch - 88ms/step
Epoch 8/50
115/115 - 10s - loss: 0.7204 - accuracy: 0.5574 - val_loss: 0.7485 - val_accuracy: 0.5243 - 10s/epoch - 85ms/step
Epoch 9/50

100%|██████████| 5/5 [3:26:41<00:00, 2480.22s/it]



Saving Test Accuracy...
Done...!!


0